In [6]:
###-----------------
### Import Libraries
###-----------------

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

from collections.abc import Callable
from typing import Literal

import tensorflow as tf
from datetime import datetime

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [7]:
# gpus = tf.config.list_physical_devices('GPU')

# try:
#     for g in gpus:
#         tf.config.experimental.set_memory_growth(g, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print (len(gpus), 'Phusical GPUs', len(logical_gpus), 'Logical GPUs')
# except:
#     print ('invalid device')

In [8]:
###----------------
### Some parameters
###----------------

inpDir = '/home/dai/Documents/DNN/7.DNN/input'
outDir = '../output'
modelDir = '../models'
subDir = 'flower_photos'
altName = 'cnn_base'
RANDOM_STATE = 24 # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 1000 # number of epochs
ALPHA = 0.001 # learning rate
NUM_SAMPLES = 1280 # How many samples we want to generate
NOISE = 0.2 # Noise to be introduced in the data
TEST_SIZE = 0.2
BATCH_SIZE = 8
PATIENCE = 25
FACTOR_LR = .5
LR_PATIENCE = 2
IMG_HEIGHT = 200
IMG_WIDTH = 200

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [9]:
data_dir = os.path.join(inpDir, subDir)
data_dir

'/home/dai/Documents/DNN/7.DNN/input/flower_photos'

In [10]:
os.listdir(data_dir)

['LICENSE.txt', 'daisy', 'roses', 'tulips', 'dandelion', 'sunflowers']

In [11]:
def fn_plot_label(tr_ds,ts_ds):
    plt.figure(figsize=(15,6)) #instantiate the figure
    plt.subplot(1,2,1)   # first out of 2
    train_labels=tf.concat([lbl for img,lbl in tr_ds],axis=0).numpy() # get the labels
    unique,_,counts=tf.unique_with_counts(train_labels)  # get counts
    # print(unique)
    # plt.bar(unique,counts,align="center",color="DarkBlue") #barplot
    plt.bar(range(len(unique)),counts,align="center",color="DarkBlue") #barplot counts
    plt.xticks(range(len(unique)),[class_names[i.numpy()] for i in unique])
    plt.title("Training Set")

    plt.subplot(1,2,2)
    test_labels=tf.concat([lbl for img,lbl in ts_ds],axis=0).numpy()
    unique,_,counts=tf.unique_with_counts(test_labels)
    plt.bar(range(len(unique)),counts,align="center",color="Orange")
    plt.xticks(range(len(unique)),[class_names[i.numpy()] for i in unique])
    plt.title("Test Set")

## using flowers data

In [12]:
#converting to dataset from directory


In [13]:
os.listdir(data_dir)


['LICENSE.txt', 'daisy', 'roses', 'tulips', 'dandelion', 'sunflowers']

In [14]:
#creating training data
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, #path to the directory
    validation_split=TEST_SIZE,  # what ratio of validation set
    subset='training', #purpose
    seed=RANDOM_STATE,
    image_size=[IMG_HEIGHT,IMG_WIDTH],  ## @@ WHAT!
    batch_size=BATCH_SIZE
)

#create Test data
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, #path to the directory
    validation_split=TEST_SIZE,  # what ratio of validation set
    subset='validation', #purpose
    seed=RANDOM_STATE,
    image_size=[IMG_HEIGHT,IMG_WIDTH],  ## @@ WHAT!   # the framework will sqeeze the images to this resolution by itself
    batch_size=BATCH_SIZE
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


2023-12-16 17:30:37.007853: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [15]:
#is it picking class names
class_names=train_ds.class_names
class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [16]:
input_shape=(IMG_HEIGHT,IMG_WIDTH,3)

## Random Zoom

In [17]:
# what do you think can be taken as a starting point?
# output number is frozen = num_classes
# approach - Let's start from the base (last conv layer) say (4*4) then move back based on our architecture we want to build and reach the start point from there, take that size as input
# Other approaches - mode of all image sizes, min images's size
# if images are of lesser dimensions then random zoom should resize them to our required size


# conv layer is kind of averaging over th eimage with filter and summing that up
# max pool tells what is most significant you have

In [18]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation((-.3,0.3))
])

In [19]:
densenet_model = tf.keras.applications.DenseNet169(weights='imagenet',include_top=False,input_shape=(IMG_HEIGHT,IMG_WIDTH,3))

51877672/51877672 [==============================] - 5s 0us/step


In [20]:
for layer in densenet_model.layers:
  layer.trainable = False

In [21]:
densenet_model.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 206, 206, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 100, 100, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                        

 conv2_block4_1_relu (Activatio  (None, 50, 50, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_2_conv (Conv2D)   (None, 50, 50, 32)   36864       ['conv2_block4_1_relu[0][0]']    
                                                                                                  
 conv2_block4_concat (Concatena  (None, 50, 50, 192)  0          ['conv2_block3_concat[0][0]',    
 te)                                                              'conv2_block4_2_conv[0][0]']    
                                                                                                  
 conv2_block5_0_bn (BatchNormal  (None, 50, 50, 192)  768        ['conv2_block4_concat[0][0]']    
 ization)                                                                                         
          

In [22]:
model= tf.keras.Sequential([tf.keras.layers.Rescaling(1/255.), #convert b/w 0 and 1
                            # data_augmentation,
                            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                            tf.keras.layers.RandomRotation((-.3,0.3)),
                            # #conv part
                            #     #input (170x170x3)
                            # tf.keras.layers.Conv2D(8,(3,3)),  # 168*168*8
                            # tf.keras.layers.BatchNormalization(),
                            # tf.keras.layers.Activation('relu'),
                            # # tf.keras.layers.Dropout(rate = .1),
                            # tf.keras.layers.MaxPool2D(4,4), # 42x42x8

                            # # tf.keras.layers.Conv2D(16,(3,3),activation='relu'),  #
                            # # tf.keras.layers.MaxPool2D(2,2), #

                            # tf.keras.layers.Conv2D(32,(3,3)),  # 40*40*32
                            # tf.keras.layers.BatchNormalization(),
                            # tf.keras.layers.Activation('relu'),
                            # # tf.keras.layers.Dropout(rate = .1),
                            # tf.keras.layers.MaxPool2D(4,4), # 10x10x32

                            # # tf.keras.layers.Conv2D(64,(3,3),activation='relu'),  #
                            # # tf.keras.layers.MaxPool2D(2,2), #

                            # tf.keras.layers.Conv2D(128,(3,3)),  # 8*8*128
                            # tf.keras.layers.BatchNormalization(),
                            # tf.keras.layers.Activation('relu'),
                            # tf.keras.layers.Dropout(rate = .1),
                            # tf.keras.layers.MaxPool2D(8,8), # 1x1x128

                            # # tf.keras.layers.Conv2D(256,(3,3),activation='relu'),  #
                            #Head part
                            tf.keras.layers.Flatten() , #Flatten
                            tf.keras.layers.Dense(1024, activation='relu'), #Dense 1  #256
                            tf.keras.layers.Dropout(rate = 0.1),
                            tf.keras.layers.Dense(5) #output  #Head
                            ])

#   accuracy: 0.7984

In [23]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim=tf.keras.optimizers.Adam(learning_rate=ALPHA)
model.compile(optimizer=optim, loss=loss_fn, metrics=['accuracy'])

In [24]:
checkpointPath = '/outputs/model'+os.path.join(modelDir,altName)+'/van'

mcp_point = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointPath,
                                                       save_weights_only = True,
                                                       save_best_only = True,
                                                       monitor = 'val_loss',
                                                       verbose = 1
)

es_point = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=PATIENCE,
                                                        verbose=1,
                                                        restore_best_weights=True,
                                                        start_from_epoch=0,
)

lr_decay = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                factor=FACTOR_LR,
                                                patience=LR_PATIENCE,
                                                verbose=1,
                                                mode='auto'
)

In [25]:
checkpointPath

'/outputs/model../models/cnn_base/van'

In [26]:
history=model.fit(train_ds,validation_data=test_ds,
                  epochs=EPOCHS,
                  verbose=1,
                  batch_size=BATCH_SIZE,
                  callbacks=[es_point,lr_decay,mcp_point])

Epoch 1/1000


2023-12-16 17:30:51.472110: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2936]
	 [[{{node Placeholder/_4}}]]
2023-12-16 17:30:51.473207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2936]
	 [[{{node Placeholder/_4}}]]


367/367 [==============================] - ETA: 0s - loss: 23.7025 - accuracy: 0.3219

2023-12-16 17:35:20.723751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [734]
	 [[{{node Placeholder/_0}}]]
2023-12-16 17:35:20.724201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [734]
	 [[{{node Placeholder/_4}}]]


PermissionDeniedError: /outputs; Permission denied

In [ ]:
model.summary()

In [ ]:
res_df=pd.DataFrame(history.history)
res_df

In [ ]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

    ###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def plot_tf_hist(hist_df):
    '''    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12

    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]

    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(y1.capitalize(),
                           hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)

    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(y2.capitalize(),
                           hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow


    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')

    plt.tight_layout()

In [ ]:
plot_tf_hist(res_df)

## Testing preds

In [ ]:
test_loss,test_acc=model.evaluate(test_ds,verbose=2)

In [ ]:
res_df[res_df['val_loss'] == res_df['val_loss'].min()]